In [1]:
import pennylane as qml
from jax import numpy as np
import jax

In [3]:
dev1 = qml.device("lightning.qubit", wires =1) #wires is basically the number of subsystems to initialise the device we are calling, in this case only a singular qubit

In [4]:
#QNodes: Abstract encapsulation of a quantum function, described by a quantum circuit. They are bound to a singular particular quantum device, which is used to evaluate expectation and variance of this circuit

In [6]:
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

In [7]:
params = np.array([0.54, 0.12])
print(circuit(params))

0.85154057


In [8]:
dcircuit = jax.grad(circuit, argnums = 0)

In [9]:
print(dcircuit(params))

[-0.5104387  -0.10267819]


In [10]:
@qml.qnode(dev1)
def circuit2(phi1, phi2):
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))

In [11]:
phi1 = np.array(0.54)
phi2 = np.array(0.12)

In [12]:
dcircuit = jax.grad(circuit2, argnums = [0,1])

In [13]:
print(dcircuit(phi1,phi2))

(Array(-0.5104387, dtype=float32), Array(-0.10267819, dtype=float32))


In [14]:
#Optimising two circuit parameters via gradient descent to minimize cost function

In [15]:
def cost(x):
    return circuit(x)

In [16]:
init_params = np.array([0.011,0.012])

In [18]:
print(cost(init_params)) #very close to 1

0.9998675


In [20]:
import jaxopt

In [21]:
opt = jaxopt.GradientDescent(cost, stepsize = 0.4, acceleration = False)

In [22]:
steps = 50

In [23]:
params = init_params

In [24]:
opt_state = opt.init_state(params)

In [26]:
for i in range(steps):
    params,opt_state = opt.update(params, opt_state)
    if (i+1)%5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i+1, cost(params)))

Cost after step     5: -1.0000000
Cost after step    10: -1.0000000
Cost after step    15: -1.0000000
Cost after step    20: -1.0000000
Cost after step    25: -1.0000000
Cost after step    30: -1.0000000
Cost after step    35: -1.0000000
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000


In [27]:
print("Optimised rotation angles: {}".format(params))

Optimised rotation angles: [7.1526556e-18 3.1415925e+00]
